<a href="https://colab.research.google.com/github/lav162329/product-category-classifier/blob/main/notebooks/02_feature_engineering_and_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook is dedicated to the application of advanced **Feature Engineering** techniques and the comparison of multiple Machine Learning algorithms. The primary goal is to create an optimal, robust model for automated product classification.

This notebook builds directly upon the data cleansing and exploratory insights gained from `01_data_exploration.ipynb`.

---

## 📥 Initialization and Data Loading

In [1]:
# Initialization and Data Loading

import pandas as pd
import numpy as np
import re
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from scipy.sparse import hstack
import warnings
warnings.filterwarnings('ignore') # Suppress warnings for cleaner output

# Load the data and perform minimal essential cleaning (reproduction of initial steps)
url = "https://raw.githubusercontent.com/lav162329/product-category-classifier/main/data/products.csv"
df = pd.read_csv(url)

# Essential cleaning steps
df.columns = df.columns.str.strip()
df.dropna(subset=['Product Title', 'Category Label'], inplace=True)
df['Product Title'] = df['Product Title'].astype(str).str.lower()
df.reset_index(drop=True, inplace=True)

print(f"Data for modeling loaded and cleaned. Shape: {df.shape}")
print("\nReady to proceed with Feature Engineering.")

Data for modeling loaded and cleaned. Shape: (35096, 8)

Ready to proceed with Feature Engineering.
